In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.1),
             loss=tf.losses.categorical_crossentropy,
             metrics=[tf.metrics.categorical_accuracy])

In [ ]:
train_x = np.random.random((6000, 72))
train_y = np.random.random((6000, 10))
model.fit(train_x, train_y, epochs=5)

In [ ]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)

In [ ]:
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

In [ ]:
result = model.predict(test_x, batch_size=32)
np.shape(result), result

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [ ]:
model = MyModel(10)
model.compile(optimizer=tf.optimizers.RMSprop(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5)

In [ ]:
class MyLayer(layers.Layer):
    def __init__(self, out_dim, **kwargs):
        self.out_dim = out_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.out_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape, 
                          initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.out_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.out_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model = tf.keras.Sequential([MyLayer(10), 
                             layers.Activation('softmax')])
model.compile(optimizer=tf.optimizers.RMSprop(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5, batch_size=16)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks)

In [ ]:
model = tf.keras.models.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer=tf.optimizers.Adam(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5)
model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./weights/weight.h5')
model.load_weights('./weights/weight.h5')

In [ ]:
import json
import pprint
json_str = model.to_json()
# pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)

yaml_str = model.to_yaml()
# print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

In [ ]:
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(72,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

import os
os.mkdir('models')
model.save('./models/model.h5')
fresh_model = tf.keras.models.load_model('./models/model.h5')

In [ ]:
estimator = tf.keras.estimator.model_to_estimator(model)